In [1]:
import sys
import time
import warnings
from math import ceil
from multiprocessing import Pool

import astropy.units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from scipy import stats
import pandas as pd

warnings.filterwarnings('ignore')

# local code imports
sys.path.append('code/')
from panstarrs import panstarrs_get_lightcurves
from gaia_functions import Gaia_get_lightcurve
from HCV_functions import HCV_get_lightcurves
from icecube_functions import icecube_get_lightcurve
from sample_selection import get_lamassa_sample, get_macleod16_sample, get_ruan_sample, get_macleod19_sample, get_sheng_sample, get_green_sample, get_lyu_sample, get_lopeznavas_sample, get_hon_sample, get_yang_sample,get_SDSS_sample, get_paper_sample, clean_sample,noclean_sample,TDE_id2coord
from data_structures import MultiIndexDFObject
from heasarc_functions import HEASARC_get_lightcurves
from TESS_Kepler_functions import TESS_Kepler_get_lightcurves
from WISE_functions import WISE_get_lightcurves
from ztf_functions_old import ZTF_get_lightcurve
import os
import re
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm

from utils import *

from tqdm import tqdm

try:
    import umap
    import umap.plot
except:
    !pip install umap-learn[plot]

plt.style.use('bmh')

def build_sample():
    '''Putting together a sample of SDSS quasars, WISE variable AGNs,
    TDEs, Changing look AGNs, .. coordinates from different 
    papers.'''
    
    coords =[]
    labels = []

    # Variable AGN sample from Ranga/Andreas:
    VAGN = pd.read_csv('data/WISE_MIR_variable_AGN_with_PS1_photometry_and_SDSS_redshift.csv')
    vagn_coords = [SkyCoord(ra, dec, frame='icrs', unit='deg') for ra, dec in zip(VAGN['SDSS_RA'], VAGN['SDSS_Dec'])]
    vagn_labels = ['WISE-Variable' for ra in VAGN['SDSS_RA']]
    coords.extend(vagn_coords)
    labels.extend(vagn_labels)    

    num_normal_QSO = 100 
    get_SDSS_sample(coords, labels, num_normal_QSO)

    get_paper_sample('2015ApJ...810...14A','FermiBL',coords,labels)
    get_paper_sample('2019A&A...627A..33D','DeCicco19',coords,labels)
    get_paper_sample('2022ApJ...933...37W','Galex variable 22',coords,labels)
    get_paper_sample('2020ApJ...896...10B','Palomar variable 20',coords,labels)

    #remove duplicates from the list if combining multiple references
    coords_list, labels_list = noclean_sample(coords, labels) #!!!!
    print('final sample: ',len(coords))
    return coords_list,labels_list


In [3]:
coords,labels = build_sample()
#dflc = parallel_lc(coords,labels)


SDSS Quasar: 100
number of sources added from FermiBL :1870
number of sources added from DeCicco19 :299
number of sources added from Galex variable 22 :48
number of sources added from Palomar variable 20 :20
without duplicates removal, sample size: 3104
final sample:  3104
